In [1]:
# !pip install plotly
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats

### Q1: 

Exploring gender difference in practicing as a “Sole Proprietor”

In [2]:
npi = pd.read_csv("data/npidata_pfile_20050523_20191013.csv")
npi.head()

,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,Provider Business Practice Location Address State Name,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number State Code_1,Is Sole Proprietor
0,1.0,WIEBE,DAVID,NE,M,207X00000X,NE,X
1,1.0,PILCHER,WILLIAM,FL,M,207RC0000X,FL,N
2,2.0,NaN,NaN,NC,NaN,251G00000X,NC,NaN
3,1.0,SMITSON,HAROLD,TX,M,2085R0202X,TX,N
4,1.0,GRESSOT,LAURENT,TX,M,174400000X,TX,N


In [3]:
print(type(npi))

<class 'pandas.core.frame.DataFrame'>


In [4]:
# check the values of the gender and sole proprietor columns
print(npi['Provider Gender Code'].unique())
print(npi['Is Sole Proprietor'].unique())

['M' nan 'F']
['X' 'N' nan 'Y']


In [5]:
# filter out the gender and sole proprietor columns
npisole= npi[['Provider Gender Code','Is Sole Proprietor']]

# delete all the nan and other types
npisole = npisole.dropna(axis=0,how='any')

In [6]:
# check the values of the gender and sole proprietor columns
print(npisole['Provider Gender Code'].unique())
print(npisole['Is Sole Proprietor'].unique())

['M' 'F']
['X' 'N' 'Y']


In [7]:
npisole = npisole[npisole['Is Sole Proprietor']!= 'X']

# check the values of the gender and sole proprietor columns
print(npisole['Provider Gender Code'].unique())
print(npisole['Is Sole Proprietor'].unique())

['M' 'F']
['N' 'Y']


In [11]:
#npisole.groupby['Provider Gender Code']
a = npisole.groupby(by=['Provider Gender Code','Is Sole Proprietor'])['Is Sole Proprietor'].count()
print(a)

Provider Gender Code  Is Sole Proprietor
F                     N                     213180
                      Y                      52216
M                     N                     333115
                      Y                      97690
Name: Is Sole Proprietor, dtype: int64


In [12]:
obs = pd.DataFrame([[213180,52216], [333115,97690]])
new_col = ['N','Y']
obs.columns=new_col
obs.index=['F','M']

print(obs)

fisher_result = stats.fisher_exact(obs)
print(fisher_result)

        N      Y
F  213180  52216
M  333115  97690
(1.1972884103650476, 4.55301455363275e-194)


### Q2: 

Gender differences in choosing practices

In [41]:
npi.head()

,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,Provider Business Practice Location Address State Name,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number State Code_1,Is Sole Proprietor
0,1.0,WIEBE,DAVID,NE,M,207X00000X,NE,X
1,1.0,PILCHER,WILLIAM,FL,M,207RC0000X,FL,N
2,2.0,NaN,NaN,NC,NaN,251G00000X,NC,NaN
3,1.0,SMITSON,HAROLD,TX,M,2085R0202X,TX,N
4,1.0,GRESSOT,LAURENT,TX,M,174400000X,TX,N


In [31]:
# check the values of the gender and ealthcare Provider Taxonomy Code
print(npi['Provider Gender Code'].unique())
#print(npi['Healthcare Provider Taxonomy Code_1'].unique())

['M' nan 'F']


In [33]:
# filter out the gender and ealthcare Provider Taxonomy Code to find the four provider codes
#  1) low risk: Obstetrics & Gynecology -207V00000X, “Pediatrics”-208000000X
#  2) high risk: Surgery - 208600000X   Orthopaedic Surgery - 207X00000X
npirisk = npi.loc[npi['Healthcare Provider Taxonomy Code_1'].isin (['207V00000X', '208000000X', '208600000X ', '207X00000X'])]
npirisk= npirisk[['Provider Gender Code','Healthcare Provider Taxonomy Code_1']]

# delete all the nan and other types
npirisk = npirisk.dropna(axis=0,how='any')

npirisk.head()

,Provider Gender Code,Healthcare Provider Taxonomy Code_1
0,M,207X00000X
9,F,208000000X
11,F,207V00000X
13,M,207X00000X
20,M,207X00000X


In [34]:
# check the values of the gender and ealthcare Provider Taxonomy Code
print(npirisk['Provider Gender Code'].unique())

['M' 'F']


In [35]:
npigenderlow = npirisk[npirisk['Healthcare Provider Taxonomy Code_1'].isin(['207V00000X','208000000X'])]

b = npigenderlow.groupby(by=['Provider Gender Code'])['Healthcare Provider Taxonomy Code_1'].size()
print(b)

Provider Gender Code
F    23715
M    22254
Name: Healthcare Provider Taxonomy Code_1, dtype: int64


In [36]:
npigenderhigh = npirisk[npirisk['Healthcare Provider Taxonomy Code_1'].isin(['208600000X ', '207X00000X'])]

c = npigenderhigh.groupby(by=['Provider Gender Code'])['Healthcare Provider Taxonomy Code_1'].size()
print(c)

Provider Gender Code
F      472
M    11910
Name: Healthcare Provider Taxonomy Code_1, dtype: int64


In [56]:
obs2 = pd.DataFrame([[23715,472], [22254,11910]])
new_col = ['Low','High']
obs2.columns=new_col
obs2.index=['F','M']

print(obs2)

     Low   High
F  23715    472
M  22254  11910


In [55]:
print(472+11910)
# 45969

12382


In [40]:
fisher_result = stats.fisher_exact(obs2)
print(fisher_result)

(26.889628868853134, 0.0)


### Q3: 

The density of national MRI centers by state population

In [42]:
# take a look of data set
npi.head()

,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,Provider Business Practice Location Address State Name,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number State Code_1,Is Sole Proprietor
0,1.0,WIEBE,DAVID,NE,M,207X00000X,NE,X
1,1.0,PILCHER,WILLIAM,FL,M,207RC0000X,FL,N
2,2.0,NaN,NaN,NC,NaN,251G00000X,NC,NaN
3,1.0,SMITSON,HAROLD,TX,M,2085R0202X,TX,N
4,1.0,GRESSOT,LAURENT,TX,M,174400000X,TX,N


In [46]:
# 这里的nipdata就是npi
entity=npi.loc[npi['Entity Type Code']==2]
mri=entity.loc[entity['Healthcare Provider Taxonomy Code_1']=='261QM1200X']
mri1=mri.loc[mri['Provider Business Practice Location Address State Name']!='GU']
mri1=mri1.loc[mri1['Provider Business Practice Location Address State Name']!='PR']

mri1.head()

,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,Provider Business Practice Location Address State Name,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number State Code_1,Is Sole Proprietor
86,2.0,NaN,NaN,OH,NaN,261QM1200X,OH,NaN
137,2.0,NaN,NaN,GA,NaN,261QM1200X,GA,NaN
155,2.0,NaN,NaN,TX,NaN,261QM1200X,NaN,NaN
176,2.0,NaN,NaN,WI,NaN,261QM1200X,NaN,NaN
196,2.0,NaN,NaN,TX,NaN,261QM1200X,NaN,NaN


In [47]:
mri1.rename(columns={'Provider Business Practice Location Address State Name':'state'}, inplace=True)

mri1.head()

,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,state,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number State Code_1,Is Sole Proprietor
86,2.0,NaN,NaN,OH,NaN,261QM1200X,OH,NaN
137,2.0,NaN,NaN,GA,NaN,261QM1200X,GA,NaN
155,2.0,NaN,NaN,TX,NaN,261QM1200X,NaN,NaN
176,2.0,NaN,NaN,WI,NaN,261QM1200X,NaN,NaN
196,2.0,NaN,NaN,TX,NaN,261QM1200X,NaN,NaN


In [64]:
mridensity=mri1.groupby('state')['Healthcare Provider Taxonomy Code_1']\
.count().reset_index(name='count')\
.sort_values(['count'], ascending=False) #group by yun

#mridensity.head()
print(mridensity)

   state  count
6     FL     69
38    TX     57
10    IL     21
31    OH     20
3     CA     20
15    MA     17
27    NJ     13
34    PA     12
30    NY     11
0     AL     10
32    OK     10
7     GA     10
1     AR      9
20    MO      9
18    MI      8
4     CO      8
11    IN      7
42    WI      7
14    LA      7
5     CT      6
16    MD      6
19    MN      5
12    KS      5
25    NE      5
13    KY      5
2     AZ      5
41    WA      4
33    OR      4
40    VA      4
23    NC      4
8     IA      4
36    SC      4
37    TN      3
22    MT      3
29    NV      3
28    NM      3
17    ME      3
9     ID      3
35    RI      2
24    ND      2
26    NH      1
39    UT      1
21    MS      1
43    WV      1


In [65]:
print(mridensity.shape[0])

44


In [67]:
population=[21299325,28701845,39557045,12741080,8908520,9995915,19542209,11689442,6902149,5611179,4659978,12807060,10519475,6042718,6691878,3943079,7171646,4887871,5695564,6126452,3013825,10383620,6770010,8517685,5813568,7535591,4468402,3156145,3572665,1356458,4190713,5084127,2986530,2095428,2911505,1929268,1754208,1338404,3034392,577737,967171,1062305,760077,3161105]

print(len(population))

44


In [68]:
population=[21299325,28701845,39557045,12741080,8908520,9995915,19542209,11689442,6902149,5611179,4659978,12807060,10519475,6042718,6691878,3943079,7171646,4887871,5695564,6126452,3013825,10383620,6770010,8517685,5813568,7535591,4468402,3156145,3572665,1356458,4190713,5084127,2986530,2095428,2911505,1929268,1754208,1338404,3034392,577737,967171,1062305,760077,3161105]

mridensity['population']=population

mridensity.head()

,state,count,population
6,FL,69,21299325
38,TX,57,28701845
10,IL,21,39557045
31,OH,20,12741080
3,CA,20,8908520


In [76]:
mridensity['mridensity']=round(mridensity['count']/mridensity['population']*1000000,2)

mridensity.head(7)

,state,count,population,mridensity
6,FL,69,21299325,3.24
38,TX,57,28701845,1.99
10,IL,21,39557045,0.53
31,OH,20,12741080,1.57
3,CA,20,8908520,2.25
15,MA,17,9995915,1.70
27,NJ,13,19542209,0.67


In [70]:
fig = go.Figure(data=go.Choropleth(
    locations=mridensity['state'], # Spatial coordinates
    z = mridensity['mridensity'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Mri density per 1M people per state",
))

In [74]:
fig.update_layout(
    title_text = 'Mri density per 1M people per state',
    geo_scope='usa', # limite map scope to USA
)

In [73]:
#fig.show()
#plotly.offline.plot(fig, filename='Mri density per 1M people per state.html')